<a href="https://colab.research.google.com/github/MinsooKwak/Study/blob/main/LLM/Transformer_XL/Transformer_XL_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Contribution**

BERT, GPT2를 포함해 기존 Transformer 모델은 "Fixed Length" 고정된 길이의 context를 갖는다는 한계가 있었다.

Transformer XL 모델은 Fixed Length context의 한계를 넘어서기 위해 고안된 최초의 모델이다.

기존에 최대 1000개의 token을 생성할 수 있었다면, Transformer XL은 1억개의 token을 생성할 수 있다.

### **Transformer XL Task**

- 생성
- 분류

## Transformer XL classification

**Label Classification**

1) Single label classification
  - 각 데이터 포인트에 하나의 label만 할당

2) Multi-class classification
  - 하나의 데이터 포인트가 여러개의 label 가질 수 있음

In [1]:
#!pip install transformers

In [2]:
#!pip install sacremoses

In [3]:
import torch
from transformers import AutoTokenizer, TransfoXLForSequenceClassification

**사전 훈련된 모델(pre-trained model) load**

In [4]:
tokenizer = AutoTokenizer.from_pretrained('transfo-xl-wt103') # transformer에서 AutoTokenizer 가져옴
model = TransfoXLForSequenceClassification.from_pretrained('transfo-xl-wt103')  # hf에서 가져옴

Some weights of TransfoXLForSequenceClassification were not initialized from the model checkpoint at transfo-xl-wt103 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Single Label Classification

In [5]:
input_str = input("")

My name is Minsoo


In [6]:
input_str

'My name is Minsoo'

In [7]:
inputs = tokenizer(input_str, return_tensors='pt')  # pytorch # tensorflow도 사용 가능

In [8]:
inputs

{'input_ids': tensor([[1162,  237,   23,   24]])}

token화 되어 있어 정수로 표현되고 있음

모델에 전달되어 predicted_class_id로 변환됨

In [9]:
with torch.no_grad():
  # logits : 예측 헤드로 전달되는 모든 값
  # .logits : 예측된 클래스를 찾는데 사용됨
  logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()

In [10]:
predicted_class_id

1

In [11]:
model.config.id2label # 원하는 경우 레이블 전달 가능

{0: 'LABEL_0', 1: 'LABEL_1'}

**Fine-tuing**

- 현재 단계에서는 실행하지 않음

In [12]:
#num_labels = len(model.config.id2label)
#model = TransfoXLForSequenceClassification.from_pretrained('transfo-xl-wt103', num_labels=num_labels)

In [13]:
#models = torch.tensor([1])
#loss = model(**inputs, labels=labels).loss

### Multi Label Classification

In [14]:
tokenizer = AutoTokenizer.from_pretrained('transfo-xl-wt103')
model = TransfoXLForSequenceClassification.from_pretrained('transfo-xl-wt103',
                                                           problem_type='multi_label_classification')

Some weights of TransfoXLForSequenceClassification were not initialized from the model checkpoint at transfo-xl-wt103 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


- Single과 다른 점은 pretrained model 가져올 때 multi_label_classification 설정하는 것

In [21]:
input_str = input("")

I was going with my dog


In [22]:
inputs = tokenizer(input_str, return_tensors='pt')

In [23]:
with torch.no_grad():
  logits = model(**inputs).logits

In [24]:
pretrained_class_ids = torch.arange(0, logits.shape[-1])[torch.sigmoid(logits).squeeze(dim=0)>0.5]

In [25]:
pretrained_class_ids

tensor([0, 1])

- 하나의 label만 있을 수 있고, 여러개 있을 수 있고
- 예측된 레이블이 없을 수도 있음

**Fine Tuning**

In [ ]:
'''
num_labels = len(model.config.id2label)
model = TransfoXLForSequenceClassification.from_pretrained('transfo-xl-wt103',
                                                           num_labels=num_labels,
                                                           problem_type='multi_label_classification')
                                                           '''

In [ ]:
#labels = torch.sum(torch.nn.functional.one_hot(pretrained_class_ids[None,:].clone(), num_class=num_labels),dim=1).to(torch.float)